In [19]:
from numpy import *
import time
import sys
import scipy.io

print("enter file name: ", end="")
fil = input()  

fil= fil +".mat"
print("Reading data from file '" + fil + "'")

if fil==None:
    print('File '+repr(fil)+' does n ot exist')
    exit()

print('Datafile: '+repr(fil))
mat=scipy.io.loadmat(fil,squeeze_me=True)

c=mat['c']
A=mat['A']
b=mat['b']
bix=asarray(mat['bix'],dtype=int)
zcheat=mat['zcheat']
if 'xcheat' in mat:
    xcheat=mat['xcheat']
else:
    xcheat=0

bix=bix-1
[m,n] = shape(A)
print('Rows: '+repr(m)+' cols: '+repr(n))

print('c:',c)
print('b:',b)
print('A:',A)
print('bix:',bix)
print('xcheat:',xcheat)
print('zcheat:',zcheat)


enter file name: Reading data from file 'lab1-small.mat'
Datafile: 'lab1-small.mat'
Rows: 23 cols: 35
c: [0.2  0.12 0.24 0.12 0.2  0.12 0.24 0.12 0.2  0.12 0.24 0.12 0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.  ]
b: [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0. 10000.  6000.  8000.  6000.
 10000.  4000.  5000.]
A: [[-2. -0. -4.  2. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-4.  1. -6. -2. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [ 1. -3.  1.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-4. -2. -2. -5. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0

In [18]:
from ctypes import sizeof
import numpy as np
import time
import sys

rc_min = np.nanmin(c)


t1=time.time()

[m,n] = np.shape(A)
print( 'Rows: '+repr(m)+' cols: '+repr(n))
i = 0
# Create nix
nix=np.setdiff1d(range(n), bix) 

B  = A[:, bix]
N  = A[:, nix]
cB = c[bix]
cN = c[nix]

# Start iterations
iter = 0
while iter >= 0:
    iter+=1            


    inverseB = np.linalg.inv(B)
    bHatt = np.matmul(inverseB, b)
    
    y = np.transpose(np.matmul(np.transpose(cB), inverseB))

    z= np.matmul(np.transpose(b),y)
    cNhatt = (cN - np.matmul(np.transpose(N),y))

    inkvarPlats = np.argmin(cNhatt)
    # inkvar = int(np.where(cNhatt==np.nanmin(cNhatt))[0][0]) #Tidigare cKhatt min
    rc_min = cNhatt[inkvarPlats]
    inkvar = nix[inkvarPlats]

  
    # calc right-hand-sides and reduced costs
    
     
    # --------

    # calc most negative reduced cost, rc_min,
    # and index for entering variable, inkvar
    # --------


    if rc_min >= -1.0E-10:
        print ('Ready')
        iter=-1
 
        # construct solution, x, and check it
        # --------
       
        x = np.full([np.size(c)], 0.00)

        

        p = bix.argsort()
        t = bHatt[p]
        x[:len(t)] += t

        
        z = np.matmul(np.transpose(b),y)
        print("funktionsvärde: ",z)

        diffx = np.linalg.norm(x-xcheat)
        diffz = z-zcheat
        print ('xdiff: '+repr(diffx))
        print ('zdiff: '+repr(diffz))
    else:
        # calc entering column, a
        # -------- 
        ak = np.matmul(inverseB, N[:, inkvarPlats])
        if max(ak) <= 0 :
            # unbounded solution
            print('Unbounded solution!')
            iter=-1
        else:
            # calc leaving var, utgvar
            # --------
            # alkHatt= np.divide(b,np.transpose(N)[inkvarPlats]) #inkommande var kol genom b (bhat), nu bestäms utgående av det minsta
            
            alkHatt = np.divide(bHatt,ak)

            #utgvarPlats = np.argmin(alkHatt)

            utgvarPlats = np.where(alkHatt > 0, alkHatt, np.inf).argmin()

            utgvar = bix[utgvarPlats]
            



            print(' Iter: '+repr(iter)+' z: '+repr(z)+' rc: '+repr(rc_min)+' ink: '+repr(inkvar+1)+' utg: '+repr(utgvar+1))

            # make new partition
            # -------
            
           
            bix[utgvarPlats] = nix[inkvarPlats]

            B = A[:,bix]
            N = A[:,nix]
            cB = c[bix]
            cN = c[nix]
            

   

print("x: ", x)

elapsed = time.time() - t1
print('Elapsed time: '+repr(elapsed))

Rows: 23 cols: 35
 Iter: 1 z: 3840.0 rc: -0.12 ink: 34 utg: 35
 Iter: 2 z: 3810.857142857142 rc: -0.04799999999999999 ink: 28 utg: 33
Ready
funktionsvärde:  3720.0000000000005
xdiff: 63934.39718068158
zdiff: 9.094947017729282e-13
x:  [3.17857143e+03 2.50000000e+03 2.14285714e+03 2.17857143e+03
 4.99600361e-13 5.14285714e+03 8.57142857e+02 2.82142857e+03
 2.35714286e+03 2.82142857e+03 1.05714286e+04 2.74285714e+04
 3.28928571e+04 7.10714286e+03 3.42857143e+03 2.05714286e+04
 1.20000000e+04 1.60000000e+04 1.45714286e+04 9.42857143e+03
 3.01071429e+04 1.89285714e+03 1.00000000e+03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Elapsed time: 0.004416465759277344


C:\Users\marcu\AppData\Local\Temp\ipykernel_21404\40563600.py:88: RuntimeWarning: divide by zero encountered in divide
  alkHatt = np.divide(bHatt,ak)
